In [ ]:
# --------- LIBRARIES IMPORT
# Import Tweepy, official Twitter API for Python
import tweepy

# Libraries needed for Senpy
import requests
from IPython.display import Code
import json

# Import other libraries needed for the project
import seaborn
import time
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
# ----/---- LIBRARIES IMPORT

# --------- TWITTER API AUTHENTICATION
# Access details for Twitter API
consumer_key = ""
consumer_secret = ""
access_token = ""
access_token_secret = ""

# Handler
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)
# ----/---- TWITTER API AUTHENTICATION

# --------- TWITTER API CALLS SETUP
# Make an API call and get some tweets
number_of_tweets = 3200
tag = ["coronavirus"]
# ----/---- TWITTER API CALLS SETUP

# --------- TWITTER API CALLS
tweets = tweepy.Cursor(api.search, count = number_of_tweets, q = tag, tweet_mode = "extended", languages = ["en"]).items(number_of_tweets)
# ----/---- TWITTER API CALLS

# --------- SENPY SERVER
# Analyse emotion
# Endpoint information and function for Senpy API call
endpoint = 'http://senpy.gsi.upm.es/api'
def query(endpoint, **kwargs):
    res = requests.get(endpoint,
                       params=kwargs)
    if res.status_code != 200:
        raise Exception(res)
    return Code(res.text, language = kwargs.get('outformat', 'json-ld'))
# ----/---- SENPY SERVER

# --------- PANDAS DATAFRAME
# Create the dataframe
df = pd.DataFrame()
# ----/---- PANDAS DATAFRAME

# --------- PANDAS DATAFRAME FUNCTION
# define the function for cleaning up the date - from messy json to pandas dataframe
def tweet_to_df(data, timestamp):
    local_df = pd.DataFrame()
    local_df = local_df.append({'time': datetime.strptime(timestamp, '%Y-%m-%d %H:%M:%S'),
                                'tweet': data['entries'][0]['nif:isString'],
                                'fear': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][0]['onyx:hasEmotionIntensity'],
                                'amusement': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][1]['onyx:hasEmotionIntensity'],
                                'anger': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][2]['onyx:hasEmotionIntensity'],
                                'annoyance': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][3]['onyx:hasEmotionIntensity'],
                                'indifference': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][4]['onyx:hasEmotionIntensity'],
                                'joy': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][5]['onyx:hasEmotionIntensity'],
                                'awe': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][6]['onyx:hasEmotionIntensity'],
                                'sadness': data['entries'][0]['onyx:hasEmotionSet'][0]['onyx:hasEmotion'][7]['onyx:hasEmotionIntensity']},
                                ignore_index=True)
    return local_df
# ----/---- PANDAS DATAFRAME FUNCTION

# --------- ITTERATE THE TWEETS
# Iterate through all the tweet objects and add them to the emotion_json
for tweet in tweets:
    # Get the response from the senpy server
    response = query(f'{endpoint}/emotion-depechemood', input = tweet.full_text)
    # Append the data to the dataframe using the function from above
    df = df.append(tweet_to_df(json.loads(str(response)), str(tweet.created_at)))
# ----/---- ITTERATE THE TWEETS

# --------- PLOT WITH SEABORN
# Get the axes
f, axes = plt.subplots(2, 4, figsize = (20, 16))

# Set the style
seaborn.set(style="whitegrid")

# Plot the emotions
# Fear
plot1 = seaborn.scatterplot(df['time'], df['fear'], color = 'orange', ax = axes[0, 0])
line1 = seaborn.lineplot(x = 'time', y = 'fear', data = df, color = 'orange', ax = axes[0, 0]);
plot1.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Amusement
plot2 = seaborn.scatterplot(df['time'], df['amusement'], color = 'blue', ax = axes[0, 1])
line2 = seaborn.lineplot(x = 'time', y = 'amusement', data = df, color = 'blue', ax = axes[0, 1]);
plot2.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Anger
plot3 = seaborn.scatterplot(df['time'], df['anger'], color = 'red', ax = axes[0, 2])
line3 = seaborn.lineplot(x = 'time', y = 'anger', data = df, color = 'red', ax = axes[0, 2]);
plot3.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Annoyance
plot4 = seaborn.scatterplot(df['time'], df['annoyance'], color = 'green', ax = axes[0, 3])
line4 = seaborn.lineplot(x = 'time', y = 'annoyance', data = df, color = 'green', ax = axes[0, 3]);
plot4.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Indifference
plot5 = seaborn.scatterplot(df['time'], df['indifference'], color = 'grey', ax = axes[1, 0])
line5 = seaborn.lineplot(x = 'time', y = 'indifference', data = df, color = 'grey', ax = axes[1, 0]);
plot5.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Joy
plot6 = seaborn.scatterplot(df['time'], df['joy'], color = 'violet', ax = axes[1, 1])
line6 = seaborn.lineplot(x = 'time', y = 'joy', data = df, color = 'violet', ax = axes[1, 1]);
plot6.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# Awe
plot7 = seaborn.scatterplot(df['time'], df['awe'], color = 'pink', ax = axes[1, 2])
line7 = seaborn.lineplot(x = 'time', y = 'awe', data = df, color = 'pink', ax = axes[1, 2]);
plot7.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))

# sadness
plot8 = seaborn.scatterplot(df['time'], df['sadness'], color = 'black', ax = axes[1, 3])
line8 = seaborn.lineplot(x = 'time', y = 'sadness', data = df, color = 'black', ax = axes[1, 3]);
plot8.set(xlim=(min(df['time']) - timedelta(seconds = 1), max(df['time']) + timedelta(seconds = 1)), ylim = (0, 1))
# ----/---- PLOT WITH SEABORN